# Agent Skills
- Created: 27 Oct 2025
- Showcases how to load and use Agent Skills on demand

In [1]:
import os
from strictjson import parse_yaml

In [2]:
from dotenv import load_dotenv
# put your secret keys in your .env
# For instance, if you are using OpenAI, your .env should contain
# export OPENAI_API_KEY = "sk-......."
load_dotenv()

True

# Define LLMs
- LLMs take in a `system_prompt` and a `user_prompt` and outputs a `str`

In [37]:
def llm(system_prompt: str, user_prompt: str, model: str = "gpt-5", effort: str = "minimal") -> str:
    ''' Here, we use OpenAI for illustration, you can change it to your own LLM '''
    # ensure your LLM imports are all within this function
    from openai import OpenAI

    client = OpenAI()
    
    # define your own LLM here
    response = client.responses.create(
    model="gpt-5",
    reasoning={"effort": effort},
    instructions=system_prompt,
    input=user_prompt,
    )

    return response.output_text

In [38]:
llm("Generate 5 names", "boys")

'- Liam\n- Noah\n- Ethan\n- Mateo\n- Alexander'

# Basic Syntax
- `system_prompt`: Instructions for the LLM
- `user_prompt`: Content to perform instructions on
- `output_format`: Concise description-type infused dictionary to specify format LLM should output in. If type is given, should specify it as a standalone string, or after a comma (e.g. `"int"` or `"an age, int"` or `"type: int"` or `"an age, type: int"`)
- `llm`: Your llm that takes in `system_prompt` and `user_prompt` and outputs a `str`

In [5]:
# Sync
parse_yaml(system_prompt = "Give me 5 names on a topic, as well as other information", 
           user_prompt = "weather",
           output_format = {"Names": "Great sounding names, List[str]",
                            "Meanings": "Name and meaning, dict", 
                            "Chinese Meanings": "Name and meaning in chinese, dict",
                            "Lucky Name or Number": "List[Union[int, str]]",
                            "Code": "Python code to generate 5 names",
                            "Sentiment": "Enum['Happy', 'Sad', 'Other']",
                            "Sentiment on a Scale of 1 (Bored) to 5 (Excited)": "Enum[1, 2, 3, 4, 5]"},
           llm = llm)

{'Names': ['Zephyr', 'Aurore', 'Nivalis', 'Solara', 'Brontes'],
 'Meanings': {'Zephyr': 'Gentle west wind; calm, refreshing breeze',
  'Aurore': 'Dawn; the first light of morning',
  'Nivalis': 'Of the snow; wintry or snow-covered',
  'Solara': 'Of the sun; warm, bright, sunny',
  'Brontes': 'Thunder; powerful storm sound'},
 'Chinese Meanings': {'Zephyr': '西风 (xī fēng) — 温和的西风',
  'Aurore': '黎明 (lí míng) — 清晨的第一道光',
  'Nivalis': '雪之 (xuě zhī) — 与雪相关，冰雪覆盖',
  'Solara': '太阳 (tài yáng) — 温暖明亮，阳光充足',
  'Brontes': '雷声 (léi shēng) — 强劲的风暴之声'},
 'Lucky Name or Number': [7, 'Zephyr', 3, 'Solara', 9],
 'Code': 'import random\nprefixes = ["Ze", "Au", "Ni", "So", "Bron", "Ael", "Cael", "Va", "Lu", "Or"]\nmiddles = ["ph", "ro", "vi", "la", "en", "ri", "na", "lo"]\nsuffixes = ["yr", "re", "is", "a", "es", "on"]\ndef generate_name():\n    return random.choice(prefixes) + random.choice(middles) + random.choice(suffixes)\nrandom.seed(42)\nnames = []\nwhile len(names) < 5:\n    n = generate_name()\n  

# Create our Skills

In [53]:
import PyPDF2
from pathlib import Path
from typing import Callable, Dict, Any, List

pdf_reader_skill = '''### How to Use the PDF Reader Skill
This skill provides several functions to open, read, and analyze PDF files.

**Available Functions:**

1. **read_pdf(file_path: str) -> str**
   - Reads and returns all text content from the given PDF file.
   - Example:
     ```python
     text = read_pdf("example.pdf")
     print(text)
     ```

2. **read_pdf_page(file_path: str, page_number: int) -> str**
   - Extracts and returns text from a specific page (0-indexed).
   - If user keys in page 5, call page 4 instead
   - Example:
     ```python
     page_text = read_pdf_page("example.pdf", 0)
     print(page_text)
     ```

3. **pdf_info(file_path: str) -> dict**
   - Returns metadata and page count for the PDF (title, author, subject, etc.).
   - Example:
     ```python
     info = pdf_info("example.pdf")
     print(info)
     ```

**Notes:**
- Make sure the file path points to a valid `.pdf` file.
- PyPDF2 must be installed: `pip install PyPDF2`
- All functions handle file-not-found and out-of-range errors gracefully.
'''

# --- Implement Skill Functions ---
def read_pdf(file_path: str) -> str:
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    text = ""
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text() or ""
    return text.strip()

def read_pdf_page(file_path: str, page_number: int) -> str:
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        if page_number < 0 or page_number >= len(reader.pages):
            raise IndexError(f"Page {page_number} is out of range.")
        return reader.pages[page_number].extract_text() or ""

def pdf_info(file_path: str) -> dict:
    path = Path(file_path)
    if not path.exists():
        raise FileNotFoundError(f"File not found: {file_path}")

    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        info = reader.metadata
        return {
            "pages": len(reader.pages),
            "title": getattr(info, "title", None),
            "author": getattr(info, "author", None),
            "subject": getattr(info, "subject", None),
        }

In [54]:
read_pdf_page("Agent Skills.pdf", 3)

'Traditional Agent + Tools (3/4)\nAgentTool 1A: pdf_reader_text(pdf_name: str)Description: Reads a pdf with pdf_name via text fieldsTool 1B: pdf_reader_ocr(pdf_name: str)Description: Reads a pdf with pdf_name via ocrTool 2A: generate_poster(description: str)Description: Generates an image in style of a poster\nTool 1C: pdf_reader_vlm(pdf_name: str)Description: Reads a pdf with pdf_name via processing each page as image with a vlmTool Context: Use pdf_reader_text when …Use pdf_reader_ocr when…Use pdf_reader_vlm when … Tool 2B: generate_high_res(description: str)Description: Generates an image in high resTool 2A: generate_cartoon(description: str)Description: Generates an image in style of a cartoonTool Context: Use generate_poster when …Use generate_high_res when…Use generate_cartoon when … Tool UseTask'

# Create some skills to execute code

In [55]:
def execute_function(func_name: str, params: dict):
    """
    Dynamically executes a function by name with given parameters.

    Args:
        func_name (str): The name of the function to execute.
        params (dict): A dictionary of parameters to pass to the function.

    Returns:
        The result of the function call, or an error message if the function is not found.
    """
    try:
        if func_name in globals():
            func = globals()[func_name]
            return func(**params)
        else:
            return f"Error: Function '{func_name}' not found."
    except Exception as e:
        return f"Error executing function '{func_name}': {e}"


In [56]:
def reply_user(reply: str):
    '''Replies the user'''
    return reply

# Add in context for agent

In [57]:
task = "What is in page 5 of Agent Skills.pdf ?"

In [58]:
res = parse_yaml(system_prompt = f'''You are to execute the user task. 
Choose the relevant skills needed for your agent. 
Available Skills:

pdf_reader: Information regarding reading and extracting text or metadata from PDF files. Use this to find out more about what you can do to read pdfs.
image_generator: Information about prompting an image generator to generate image. Use this if you want to generate images.
''', 
           user_prompt = f"Task: {tadsk}",
           output_format = {"Skill Names": "type: list[str]"},
           llm = llm)

In [59]:
res

{'Skill Names': ['pdf_reader']}

In [60]:
skill_context = ""
if "pdf_reader" in res["Skill Names"]:
    skill_context += pdf_reader_skill

In [61]:
skill_context

'### How to Use the PDF Reader Skill\nThis skill provides several functions to open, read, and analyze PDF files.\n\n**Available Functions:**\n\n1. **read_pdf(file_path: str) -> str**\n   - Reads and returns all text content from the given PDF file.\n   - Example:\n     ```python\n     text = read_pdf("example.pdf")\n     print(text)\n     ```\n\n2. **read_pdf_page(file_path: str, page_number: int) -> str**\n   - Extracts and returns text from a specific page (0-indexed).\n   - If user keys in page 5, call page 4 instead\n   - Example:\n     ```python\n     page_text = read_pdf_page("example.pdf", 0)\n     print(page_text)\n     ```\n\n3. **pdf_info(file_path: str) -> dict**\n   - Returns metadata and page count for the PDF (title, author, subject, etc.).\n   - Example:\n     ```python\n     info = pdf_info("example.pdf")\n     print(info)\n     ```\n\n**Notes:**\n- Make sure the file path points to a valid `.pdf` file.\n- PyPDF2 must be installed: `pip install PyPDF2`\n- All functions

In [62]:
history = []

for step in range(3):
    # Do next step
    res = parse_yaml(system_prompt = f'''You are to execute the user task. 
Additional Context: ```{skill_context}```

Here are the functions you can use in the form function_name(function_parameters) -> function_return_types: function_description:
reply_user(reply: str) -> str: Reply to the user. Use this when you have completed the user task

Use only the functions provided, including those in the additional context, and nothing more.
If function has no parameters, then return an empty dictionary for Function Parameters if you want to use it.

Output Thoughts, Function Name and Function Parameters.
''', 
           user_prompt = f"Past History: {history}\nTask: {task}",
           output_format = {"Thoughts": "What to do next, type: str",
                           "Function Name": "Function name to call, type: str",
                           "Function Parameters": "Parameters for the function, in the form key: value, type: dict"},
           llm = llm)

    func_name = res["Function Name"]
    func_params = res["Function Parameters"]
    output = execute_function(func_name, func_params)
    step_summary = f"Function {func_name} executed with parameters {func_params}. Output: {output}"
    print(f"### Step {step+1}\n\n")
    print(step_summary)
    history.append(step_summary)

    # Call another additional context update here if needed

    if func_name == "reply_user": break

### Step 1


Function read_pdf_page executed with parameters {'file_path': 'Agent Skills.pdf', 'page_number': 4}. Output: Traditional Agent + Tools (4/4)
AgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTool 3ATool 3BTool 3CTool 3DTool 3ETool 3FTool 4ATool 4BTool 4CTool 4DTool 4ETool 4FContext 1Context 2Context 3Context 4It’s too much information!
Tool UseTask
### Step 2


Function read_pdf_page executed with parameters {'file_path': 'Agent Skills.pdf', 'page_number': 4}. Output: Traditional Agent + Tools (4/4)
AgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTool 3ATool 3BTool 3CTool 3DTool 3ETool 3FTool 4ATool 4BTool 4CTool 4DTool 4ETool 4FContext 1Context 2Context 3Context 4It’s too much information!
Tool UseTask
### Step 3


Function reply_user executed with parameters {'reply': 'Traditional Agent + Tools (4/4)\\nAgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTo

In [63]:
history

["Function read_pdf_page executed with parameters {'file_path': 'Agent Skills.pdf', 'page_number': 4}. Output: Traditional Agent + Tools (4/4)\nAgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTool 3ATool 3BTool 3CTool 3DTool 3ETool 3FTool 4ATool 4BTool 4CTool 4DTool 4ETool 4FContext 1Context 2Context 3Context 4It’s too much information!\nTool UseTask",
 "Function read_pdf_page executed with parameters {'file_path': 'Agent Skills.pdf', 'page_number': 4}. Output: Traditional Agent + Tools (4/4)\nAgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTool 3ATool 3BTool 3CTool 3DTool 3ETool 3FTool 4ATool 4BTool 4CTool 4DTool 4ETool 4FContext 1Context 2Context 3Context 4It’s too much information!\nTool UseTask",
 "Function reply_user executed with parameters {'reply': 'Traditional Agent + Tools (4/4)\\\\nAgentTool 1ATool 1BTool 1CTool 1DTool 1ETool 1FTool 2ATool 2BTool 2CTool 2DTool 2ETool 2FTool 3ATool 3BTool 3CTool